<a href="https://colab.research.google.com/github/Ifeeding99/tensorflow-scripts/blob/main/Vision_transformer_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 93.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorboard
    Found existing i

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

In [ ]:
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

169001437/169001437 [==============================] - 2s 0us/step


In [ ]:
learning_rate = 1e-3
lr_decay = 1e-4
batch_size = 64
epochs = 20
img_size = 72
num_heads_= 4
patch_size = 10
num_patches = (img_size // patch_size)**2
projection_dim = 64
transformers_layers = 8
mlp_head_units = [2048, 1024]
transformer_units = [
    projection_dim *2,
    projection_dim
]

In [ ]:
def mlp(x, hidden_units, dropout_rate):
  for units in hidden_units:
    x = tf.keras.layers.Dense(units, activation = tf.nn.gelu)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
  return x

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Normalization(),
    tf.keras.layers.Resizing(img_size, img_size),
    tf.keras.layers.RandomFlip(),
    tf.keras.layers.RandomRotation(factor=0.02),
    tf.keras.layers.RandomZoom(0.2)
],
name = 'data_augmentation')
data_augmentation.layers[0].adapt(X_train)

In [ ]:
class Patches(tf.keras.layers.Layer):
  def __init__(self, patch_size):
    super().__init__()
    self.patch_size = patch_size

  def call(self, images):
    batch_size = tf.shape(images)[0]
    patches = tf.image.extract_patches(
        images = images,
        sizes = [1, self.patch_size, self.patch_size, 1],
        strides = [1, self.patch_size, self.patch_size, 1],
        rates = [1,1,1,1],
        padding = 'VALID'
    )
    patch_dims = patches.shape[-1]
    patches = tf.reshape(patches, [batch_size, -1, patch_dims])
    return patches

In [ ]:
class PatchEncoder(tf.keras.layers.Layer):
  def __init__(self, num_patches, projection_dim):
    super().__init__()
    self.num_patches = num_patches
    self.projection_dim = tf.keras.layers.Dense(projection_dim)
    self.positional_embedding = tf.keras.layers.Embedding(input_dim = num_patches, output_dim = projection_dim)

  def call(self, patch):
    positions = tf.range(start=0, limit = self.num_patches, delta=1)
    encoded = self.projection_dim(patch) + self.positional_embedding(positions)
    return encoded

In [ ]:
def create_ViT_classifier():
  input_shape = (32,32,3)
  num_classes = 100
  inputs = tf.keras.Input(shape = input_shape)
  augmented = data_augmentation(inputs)
  patches = Patches(patch_size)(augmented)
  encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

  for _ in range (transformers_layers):
    x1 = tf.keras.layers.LayerNormalization()(encoded_patches)
    attention_output = tf.keras.layers.MultiHeadAttention(num_heads = num_heads_, key_dim = projection_dim, dropout = 0.1)(x1,x1)
    x2 = tf.keras.layers.Add()([attention_output, encoded_patches])
    x3 = tf.keras.layers.LayerNormalization()(x2)
    x3 = mlp(x3, hidden_units = transformer_units, dropout_rate = 0.1)
    encoded_patches = tf.keras.layers.Add()([x3,x2])

  representation = tf.keras.layers.LayerNormalization()(encoded_patches)
  representation = tf.keras.layers.Flatten()(representation)
  features = mlp(representation, hidden_units = mlp_head_units, dropout_rate = 0.5)
  logits = tf.keras.layers.Dense(num_classes)(features)
  model = tf.keras.Model(inputs = inputs, outputs = logits)
  return model

In [ ]:
def run_experiment(model):
  optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=lr_decay)
  model.compile(optimizer = optimizer,
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics = [
                    tf.keras.metrics.SparseCategoricalAccuracy(name = 'accuracy'),
                    tf.keras.metrics.SparseTopKCategoricalAccuracy(5, name = 'top-5-accuracy'),
                ])

  history = model.fit(X_train, y_train,
                      validation_data = (X_val, y_val),
                      batch_size = batch_size,
                      epochs = 20)

In [ ]:
model = create_ViT_classifier()
run_experiment(model)

Epoch 1/20
782/782 [==============================] - 84s 92ms/step - loss: 4.2832 - accuracy: 0.0598 - top-5-accuracy: 0.2008 - val_loss: 3.7133 - val_accuracy: 0.1306 - val_top-5-accuracy: 0.3692
Epoch 2/20
782/782 [==============================] - 69s 88ms/step - loss: 3.7586 - accuracy: 0.1184 - top-5-accuracy: 0.3499 - val_loss: 3.4068 - val_accuracy: 0.1856 - val_top-5-accuracy: 0.4534
Epoch 3/20
782/782 [==============================] - 69s 88ms/step - loss: 3.5023 - accuracy: 0.1599 - top-5-accuracy: 0.4224 - val_loss: 3.1726 - val_accuracy: 0.2227 - val_top-5-accuracy: 0.5076
Epoch 4/20
782/782 [==============================] - 68s 88ms/step - loss: 3.3363 - accuracy: 0.1890 - top-5-accuracy: 0.4691 - val_loss: 3.0696 - val_accuracy: 0.2512 - val_top-5-accuracy: 0.5402
Epoch 5/20
782/782 [==============================] - 68s 87ms/step - loss: 3.1985 - accuracy: 0.2181 - top-5-accuracy: 0.5046 - val_loss: 2.9439 - val_accuracy: 0.2688 - val_top-5-accuracy: 0.5736
Epoch 6/20